# Imported important modules

In [17]:
from numpy import mean
from numpy import std
from numpy.random import randn
from numpy.random import seed
from matplotlib import pyplot
import pandas as pd
from numpy import cov
from scipy.stats import pearsonr
from scipy.stats import spearmanr
from scipy.stats import kendalltau
import numpy as np
import math
import plotly.graph_objects as go
import plotly.express as px
import requests
import json
from flask import jsonify
from csv import writer
import csv
from statistics import mean
import statistics

# Making csv file

In [ ]:



url = 'http://coronavirus-tracker-api.herokuapp.com/all'
url_iso2 = 'https://restcountries.eu/rest/v2/'


response_iso2 = requests.get(url_iso2)
lst_iso2 = json.loads(response_iso2.text)

response = requests.get(url)
lst = json.loads(response.text)

dict_pop = {}
dict_iso3 = {}
dict_affected = {}
dict_deaths = {}
dict_recovered = {}

for i in lst_iso2:
    dict_pop[i['alpha2Code']] = i['population']
    dict_iso3[i['alpha2Code']] = i['alpha3Code']
    dict_affected[i['alpha2Code']] = {}
    dict_deaths[i['alpha2Code']] = {}
    dict_recovered[i['alpha2Code']] = {}
      

## INITIALIZATION ##

for i in lst['confirmed']['locations']:
    code = i['country_code']
    if code in dict_iso3:
        for j in i['history']:
            dict_affected[code][j] = 0

for i in lst['recovered']['locations']:
    code = i['country_code']
    if code in dict_iso3:
        for j in i['history']:
            dict_recovered[code][j] = 0

for i in lst['deaths']['locations']:
    code = i['country_code']
    if code in dict_iso3:
        for j in i['history']:
            dict_deaths[code][j] = 0         


## Summing up ##

for i in lst['deaths']['locations']:
    name = i['country']
    code = i['country_code']
    if code in dict_iso3:
        for j in i['history']:
            dict_deaths[code][j] = dict_deaths[code][j] + i['history'][j] 

for i in lst['recovered']['locations']:
    name = i['country']
    code = i['country_code']
    if code in dict_iso3:
        for j in i['history']:
            dict_recovered[code][j] = dict_recovered[code][j] + i['history'][j] 

for i in lst['confirmed']['locations']:
    name = i['country']
    code = i['country_code']
    if code in dict_iso3:
        for j in i['history']:
            dict_affected[code][j] = dict_affected[code][j] + i['history'][j]                         

#************************************************
# with open('Literacy_rate/Databihin.csv', 'rt') as f:
#     lines = csv.reader(f)
#     for line in lines:
#         map_iso3_latest_year[line[0]] = 2018
#         map_iso3_literacy[line[0]] = 99
#         # print(line[0], map_iso3_literacy[line[0]])
#************************************************
map_country_dataList_mx_temp = {}
map_country_dataList_mn_temp = {}
map_country_dataList_hum = {}
with open('weather_data_countries_covid19.csv') as f:
    lines = csv.reader(f)
    for line in lines:
        if line[1] == 'maxtempC':
            map_country_dataList_mx_temp[line[0]] = line[2:-1]
        elif line[1] == 'mintempC':
            map_country_dataList_mn_temp[line[0]] = line[2:-1]
        else:
            map_country_dataList_hum[line[0]] = line[2:-1]

# for i in map_country_dataList_mx_temp:
#     print(i, map_country_dataList_mx_temp[i])

with open('country_model_e.csv', 'w') as csv_file:
    csv_writer = csv.writer(csv_file)
    headers = ['Country_name', 'Country_code_iso2', 'Country_code_iso3', 'Date', 'Tot_affected', 'Tot_dead', 'Tot_recov', 'Population', 'minTempC', 'maxTempC', 'Humidity', 'newlyAffected','newlyDead','newlyRecovered']
    csv_writer.writerow(headers)
    for i in lst['confirmed']['locations']:
        name = i['country']
        code = i['country_code']
        if code in dict_iso3:
            vnt = 0
            prev = 0
            prev_d = 0
            prev_r = 0
            for j in i['history']:
                date = j + '20'
                if name in map_country_dataList_mn_temp:
                    #entry = [name, code, dict_iso3[code], date, dict_affected[code][j], dict_deaths[code][j], dict_recovered[code][j], dict_pop[code], map_country_dataList_mn_temp[name][vnt], map_country_dataList_mx_temp[name][vnt], map_country_dataList_hum[name][vnt], dict_affected[code][j]]

                    try:
                        mn_temp = map_country_dataList_mn_temp[name][vnt]
                        mx_temp = map_country_dataList_mx_temp[name][vnt]
                        hum = map_country_dataList_hum[name][vnt]
                    except:
                        mn_temp = 'null'
                        mx_temp = 'null'
                        hum = 'null'

                    try:
                        newly_aff = dict_affected[code][j] - prev
                        newly_rec = dict_recovered[code][j] - prev_r
                        newly_ded = dict_deaths[code][j] - prev_d
                    except:
                        newly_aff = dict_affected[code][j]
                        newly_rec = dict_recovered[code][j]
                        newly_ded = dict_deaths[code][j]
                    prev = dict_affected[code][j]
                    prev_r = dict_recovered[code][j]
                    prev_d = dict_deaths[code][j]
                    #newly_aff = dict_affected[code][j] - dict_affected[code][j-1]
                    ent = [name, code, dict_iso3[code], date, dict_affected[code].get(j, 0), dict_deaths[code].get(j, 0), dict_recovered[code].get(j, 0), dict_pop[code], mn_temp, mx_temp, hum, newly_aff, newly_ded, newly_rec]
                vnt = vnt + 1
                # if vnt > 60:
                #     break
                csv_writer.writerow(ent)
                if date == '4/6/2020':
                    break
    




# two variables for plotting

# my written function for pearson (  no need because scipy has built in function)

In [ ]:
def average(x):
    assert len(x) > 0
    return float(sum(x)) / len(x)

def pearson_def(x, y):
    assert len(x) == len(y)
    n = len(x)
    assert n > 0
    avg_x = average(x)
    avg_y = average(y)
    diffprod = 0
    xdiff2 = 0
    ydiff2 = 0
    for idx in range(n):
        xdiff = x[idx] - avg_x
        ydiff = y[idx] - avg_y
        diffprod += xdiff * ydiff
        xdiff2 += xdiff * xdiff
        ydiff2 += ydiff * ydiff

    return diffprod / math.sqrt(xdiff2 * ydiff2)

# A function for checking different times of correlations, mean, standard deviation and covariance

In [24]:
countrys = [] #country names
correlationp = [] ##pearson
correlations = [] #spearsman
correlationk = [] #kendall
commentpearson =[]
commentspearsman = []
commentkendall = []
mediantemp = []
mediannewlyaffected = []
pearp=[]
spearp=[]
kendallp=[]

def correl(bd):
    Bd1 = bd.iloc[:,9:10]
    Bd2 = bd.iloc[:,11:12]
    countrys.append(bd.iloc[0,0])
    
    X = [x[0] for x in Bd1.values.tolist()]  #list comprehension
    Y = [y[0] for y in Bd2.values.tolist()]
    print('data1: mean=%.3f stdv=%.3f' % (mean(X), std(X)))
    print('data2: mean=%.3f stdv=%.3f' % (mean(Y), std(Y)))
    
    mediantemp.append(statistics.median(X)) 
    mediannewlyaffected.append(statistics.median(Y)) 

    
    #print('my function pearson : %.5f' % pearson_def(X,Y))
    
    covariance = cov(X,Y)
    print('covarience value :')
    print( covariance)
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=X,y=Y,mode='markers',marker_color=Y)) # hover text goes here

    fig.update_layout(title='Temperature vs Affected', xaxis=dict(title='Temperature'),yaxis=dict(title='Total # of Affected'))
    
    fig.show()
    ##pyplot.scatter(Bd1, Bd2)
    #pyplot.xlabel("temparature")
    #pyplot.ylabel("Total Affected")
    #pyplot.show()
    alpha = 0.05 #setting my significant level 5 %
    corr, r = pearsonr(X,Y)
    print('Pearsons correlation: %.3f' % corr)
    correlationp.append('%.3f'%(corr))
    
    if r > alpha:
        print('Samples are uncorrelated (fail to reject H0) p=%.3f' % r)
        commentpearson.append('uncorrelated')
        pearp.append('%.3f'%(r))
    else:
        print('Samples are correlated (reject H0) p=%.3f' % r)
        commentpearson.append('correlated')
        pearp.append('%.3f'%(r))
        
    corr, p = spearmanr(X,Y)
    print('Spearmans correlation: %.3f' % corr)
    correlations.append('%.3f'%(corr))
    
    # interpret the significance
    
    if p > alpha:
        print('Samples are uncorrelated (fail to reject H0) p=%.3f' % p)
        commentspearsman.append('uncorrelated')
        spearp.append('%.3f'%(p))
    else:
        print('Samples are correlated (reject H0) p=%.3f' % p)
        commentspearsman.append('correlated')
        spearp.append('%.3f'%(p))
        
    
    coef, q = kendalltau(X,Y)
    print('Kendall correlation coefficient: %.3f' % coef)
    correlationk.append('%.3f'%(coef))
    # interpret the significance
    alpha = 0.05
    if q > alpha:
        print('Samples are uncorrelated (fail to reject H0) p=%.3f' % q)
        commentkendall.append('uncorrelated')
        kendallp.append('%.3f'%(q))
    else:
        print('Samples are correlated (reject H0) p=%.3f' % q)
        commentkendall.append('correlated')
        kendallp.append('%.3f'%(q))

    


# For all countries (New approach)


# Deleting duplicates

In [ ]:
#deleting duplicates
with open('country_model_e.csv','r') as in_file, open('temperaturehumidity.csv','w') as out_file:
    seen = set() # set for fast O(1) amortized lookup
    for line in in_file:
        if line in seen: continue # skip duplicate

        seen.add(line)
        out_file.write(line)

# Temaparaturehumidity has temperature and humidity of 5 days prior to covid data (covid data from 27/1/2020 to 6/4/2020) And did some cleaning (new cases 0 are removed)

In [2]:
with open('temperaturehumidity.csv', "r") as f:
    data = list(csv.reader(f))

with open('temperaturehumidity2.csv', "w") as f2:
    writer = csv.writer(f2)
    for row in data:
        if row: # only use non-empty lines (or use if len(row)>2)
            if row[8] != 'null' and row[9] != 'null' and row[10] != 'null' and row[11]!= '0':
                  writer.writerow(row)

# Preparing datasets and calling functions to find out correlations

In [25]:
df = pd.read_csv('temperaturehumidity2.csv')
dfg = df.groupby('Country_name').apply(correl)
dfg.head(20)
        
#print(dfg.size())
#for i in dfg.size():
   # print(i)
#for name, group in dfg:
    #print(name)
    #print(group)
    #print(group.size())
    #for i in range(0,13376,76):
        #df1 = df.iloc[i:i+71, 9:10]
        #df2 = df.iloc[i+5:i+76, 11:12]
        #print(df1)
        #print('the index no. is : %s' %i )
        #correl(df1,df2)
        #countrys.append(df.iloc[i,0])

#X = df.iloc[0:71, 8:9]
#Y = df.iloc[0:71, 4:5]
#Z = df.iloc[71:142, 8:9]"""

data1: mean=11.870 stdv=3.757
data2: mean=15.957 stdv=17.682
covarience value :
[[ 14.75494071  27.9486166 ]
 [ 27.9486166  326.86166008]]


Pearsons correlation: 0.402
Samples are uncorrelated (fail to reject H0) p=0.057
Spearmans correlation: 0.540
Samples are correlated (reject H0) p=0.008
Kendall correlation coefficient: 0.412
Samples are correlated (reject H0) p=0.009
data1: mean=14.724 stdv=3.628
data2: mean=13.000 stdv=8.120
covarience value :
[[13.63546798  7.46428571]
 [ 7.46428571 68.28571429]]


Pearsons correlation: 0.245
Samples are uncorrelated (fail to reject H0) p=0.201
Spearmans correlation: 0.389
Samples are correlated (reject H0) p=0.037
Kendall correlation coefficient: 0.272
Samples are correlated (reject H0) p=0.047
data1: mean=18.848 stdv=3.258
data2: mean=43.121 stdv=47.868
covarience value :
[[  10.94507576  -18.32481061]
 [ -18.32481061 2362.92234848]]


Pearsons correlation: -0.114
Samples are uncorrelated (fail to reject H0) p=0.528
Spearmans correlation: -0.142
Samples are uncorrelated (fail to reject H0) p=0.430
Kendall correlation coefficient: -0.090
Samples are uncorrelated (fail to reject H0) p=0.482
data1: mean=7.909 stdv=4.776
data2: mean=23.864 stdv=12.392
covarience value :
[[ 23.8961039   -3.39393939]
 [ -3.39393939 160.88528139]]


Pearsons correlation: -0.055
Samples are uncorrelated (fail to reject H0) p=0.809
Spearmans correlation: -0.058
Samples are uncorrelated (fail to reject H0) p=0.798
Kendall correlation coefficient: -0.041
Samples are uncorrelated (fail to reject H0) p=0.798
data1: mean=32.107 stdv=4.047
data2: mean=55.500 stdv=59.832
covarience value :
[[  16.98809524  -70.72222222]
 [ -70.72222222 3712.48148148]]


...

Pearsons correlation: 0.119
Samples are uncorrelated (fail to reject H0) p=0.596
Spearmans correlation: 0.377
Samples are uncorrelated (fail to reject H0) p=0.084
Kendall correlation coefficient: 0.283
Samples are uncorrelated (fail to reject H0) p=0.073
data1: mean=24.850 stdv=1.931
data2: mean=8.350 stdv=6.295
covarience value :
[[ 3.92368421  1.10789474]
 [ 1.10789474 41.71315789]]


Pearsons correlation: 0.087
Samples are uncorrelated (fail to reject H0) p=0.717
Spearmans correlation: 0.049
Samples are uncorrelated (fail to reject H0) p=0.838
Kendall correlation coefficient: 0.037
Samples are uncorrelated (fail to reject H0) p=0.838
data1: mean=26.378 stdv=4.658
data2: mean=6.568 stdv=4.722
covarience value :
[[22.2972973   6.86261261]
 [ 6.86261261 22.91891892]]


Pearsons correlation: 0.304
Samples are uncorrelated (fail to reject H0) p=0.068
Spearmans correlation: 0.268
Samples are uncorrelated (fail to reject H0) p=0.109
Kendall correlation coefficient: 0.193
Samples are uncorrelated (fail to reject H0) p=0.110
data1: mean=-0.393 stdv=2.540
data2: mean=9.071 stdv=8.771
covarience value :
[[ 6.69179894 -3.15608466]
 [-3.15608466 79.77248677]]


Pearsons correlation: -0.137
Samples are uncorrelated (fail to reject H0) p=0.488
Spearmans correlation: -0.129
Samples are uncorrelated (fail to reject H0) p=0.513
Kendall correlation coefficient: -0.098
Samples are uncorrelated (fail to reject H0) p=0.503


""


In [ ]:
dfg.head()

# Previous approach

# Reading datasests for specific countrys (might change later)

In [ ]:
dataset = pd.read_csv('country_model3.csv')
dataset
Bd2 = dataset.iloc[1:71, 4:5]
Bd1 = dataset.iloc[1:71, 8:9]

Br2 = dataset.iloc[72:142, 4:5]
Br1 = dataset.iloc[72:142, 8:9]

Cn2 = dataset.iloc[143:213, 4:5]
Cn1 = dataset.iloc[143:213, 8:9]

In2 = dataset.iloc[214:284, 4:5]
In1 = dataset.iloc[214:284, 8:9]

It2 = dataset.iloc[285:355, 4:5]
It1 = dataset.iloc[285:355, 8:9]


Uk2 = dataset.iloc[356:426, 4:5]
Uk1 = dataset.iloc[356:426, 8:9]

Us2 = dataset.iloc[427:497,4:5]
Us1 = dataset.iloc[427:497, 8:9]


Bangladesh

In [ ]:
correl(Bd1,Bd2)
countrys.append('Bangladesh')

India

In [ ]:
correl(In1,In2)
countrys.append('India')

USA

In [ ]:
correl(Us1,Us2)
countrys.append('United States')

United Kingdom

In [ ]:
correl(Uk1,Uk2)
countrys.append('United Kingdom')

China

In [ ]:
correl(Cn1,Cn2)
countrys.append('China')

Italy

In [ ]:
correl(It1,It2)
countrys.append('Italy')

Brazil

In [ ]:
correl(Br1,Br2)
countrys.append('Brazil')

In [4]:
len(countrys)

115

In [5]:
print(countrys)

['Afghanistan', 'Albania', 'Algeria', 'Andorra', 'Argentina', 'Armenia', 'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh', 'Belarus', 'Belgium', 'Bolivia', 'Bosnia and Herzegovina', 'Brazil', 'Brunei', 'Bulgaria', 'Burkina Faso', 'Cabo Verde', 'Cambodia', 'Canada', 'Chile', 'China', 'Colombia', 'Congo (Brazzaville)', 'Congo (Kinshasa)', 'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Cuba', 'Cyprus', 'Denmark', 'Dominican Republic', 'Ecuador', 'Egypt', 'Estonia', 'Finland', 'France', 'Georgia', 'Germany', 'Ghana', 'Greece', 'Guinea', 'Honduras', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iran', 'Iraq', 'Ireland', 'Israel', 'Italy', 'Japan', 'Jordan', 'Kazakhstan', 'Korea, South', 'Kuwait', 'Kyrgyzstan', 'Latvia', 'Lebanon', 'Lithuania', 'Luxembourg', 'Malaysia', 'Malta', 'Mauritania', 'Mauritius', 'Mexico', 'Moldova', 'Montenegro', 'Morocco', 'Netherlands', 'New Zealand', 'Nigeria', 'North Macedonia', 'Norway', 'Oman', 'Pakistan', 'Panama', 'Papua New Guinea', 'Paraguay', 'Pe

In [7]:
print(correlationp.sort())

None


In [9]:
len(correlationp)

115

In [10]:
print(correlationp)

['-0.0019', '-0.0021', '-0.0040', '-0.0058', '-0.0146', '-0.0210', '-0.0275', '-0.0544', '-0.0547', '-0.0553', '-0.0583', '-0.0782', '-0.0839', '-0.0865', '-0.1064', '-0.1139', '-0.1209', '-0.1330', '-0.1366', '-0.1697', '-0.1777', '-0.2100', '-0.2274', '-0.2461', '-0.2483', '-0.2642', '-0.2715', '-0.2816', '-0.2853', '-0.3130', '-0.3135', '-0.3181', '-0.3218', '-0.3315', '-0.3524', '-0.3989', '-0.4070', '-0.4391', '-0.4450', '-0.4574', '-0.4792', '-0.5453', '-0.5744', '-0.6593', '0.0019', '0.0077', '0.0208', '0.0217', '0.0349', '0.0360', '0.0461', '0.0621', '0.0631', '0.0773', '0.0800', '0.0825', '0.0828', '0.0866', '0.1000', '0.1085', '0.1186', '0.1195', '0.1579', '0.1595', '0.1631', '0.1838', '0.1838', '0.1840', '0.1857', '0.1889', '0.1949', '0.1966', '0.2032', '0.2192', '0.2229', '0.2230', '0.2421', '0.2446', '0.2468', '0.2478', '0.2580', '0.2592', '0.2908', '0.3022', '0.3036', '0.3077', '0.3161', '0.3235', '0.3413', '0.3536', '0.3563', '0.3634', '0.3649', '0.3680', '0.3870', '0.39

In [11]:
print(statistics.median(correlationp))

0.0866


In [12]:
correlations.sort()
correlationk.sort()

In [13]:
print(correlations)

['-0.0009', '-0.0035', '-0.0044', '-0.0216', '-0.0265', '-0.0283', '-0.0358', '-0.0386', '-0.0578', '-0.0661', '-0.0815', '-0.0826', '-0.0881', '-0.0977', '-0.1029', '-0.1075', '-0.1177', '-0.1214', '-0.1289', '-0.1423', '-0.1741', '-0.1906', '-0.2074', '-0.2171', '-0.2275', '-0.2443', '-0.2793', '-0.3034', '-0.3035', '-0.3043', '-0.3065', '-0.3193', '-0.3293', '-0.3419', '-0.3831', '-0.3836', '-0.4099', '-0.4107', '-0.4828', '-0.6000', '-0.6106', '-0.6909', '-0.7071', '-0.7644', '0.0166', '0.0174', '0.0376', '0.0417', '0.0441', '0.0490', '0.0543', '0.0579', '0.0586', '0.0785', '0.0841', '0.0899', '0.0914', '0.0930', '0.1018', '0.1224', '0.1271', '0.1287', '0.1339', '0.1414', '0.1521', '0.1672', '0.1818', '0.2198', '0.2259', '0.2297', '0.2316', '0.2416', '0.2517', '0.2623', '0.2641', '0.2676', '0.2727', '0.2758', '0.2765', '0.2773', '0.2788', '0.2803', '0.2855', '0.2977', '0.3160', '0.3496', '0.3559', '0.3620', '0.3693', '0.3766', '0.3786', '0.3854', '0.3866', '0.3892', '0.3915', '0.40

In [14]:
statistics.median(correlations)

'0.0930'

In [15]:
print(correlationk)

['-0.0050', '-0.0129', '-0.0139', '-0.0166', '-0.0172', '-0.0191', '-0.0263', '-0.0281', '-0.0365', '-0.0378', '-0.0407', '-0.0651', '-0.0682', '-0.0795', '-0.0865', '-0.0895', '-0.0978', '-0.1010', '-0.1238', '-0.1289', '-0.1346', '-0.1408', '-0.1547', '-0.1611', '-0.1979', '-0.2037', '-0.2119', '-0.2168', '-0.2266', '-0.2680', '-0.2718', '-0.2832', '-0.2843', '-0.3025', '-0.3032', '-0.3081', '-0.3322', '-0.4102', '-0.4689', '-0.5237', '-0.6199', '-0.6325', '0.0000', '0.0064', '0.0098', '0.0111', '0.0167', '0.0212', '0.0245', '0.0366', '0.0411', '0.0431', '0.0435', '0.0526', '0.0600', '0.0643', '0.0714', '0.0777', '0.0800', '0.0813', '0.0867', '0.0984', '0.1049', '0.1100', '0.1236', '0.1249', '0.1348', '0.1675', '0.1703', '0.1714', '0.1730', '0.1755', '0.1829', '0.1829', '0.1845', '0.1925', '0.1933', '0.1964', '0.2021', '0.2209', '0.2212', '0.2296', '0.2300', '0.2309', '0.2337', '0.2361', '0.2491', '0.2582', '0.2591', '0.2717', '0.2753', '0.2792', '0.2832', '0.2900', '0.2931', '0.3108

In [16]:
statistics.median(correlationk)

'0.0777'

In [14]:
print(commentpearson)

['uncorrelated', 'uncorrelated', 'uncorrelated', 'uncorrelated', 'uncorrelated', 'uncorrelated', 'correlated', 'correlated', 'uncorrelated', 'uncorrelated', 'correlated', 'uncorrelated', 'correlated', 'uncorrelated', 'uncorrelated', 'correlated', 'uncorrelated', 'uncorrelated', 'uncorrelated', 'uncorrelated', 'uncorrelated', 'correlated', 'uncorrelated', 'correlated', 'correlated', 'uncorrelated', 'uncorrelated', 'uncorrelated', 'uncorrelated', 'uncorrelated', 'correlated', 'uncorrelated', 'uncorrelated', 'uncorrelated', 'uncorrelated', 'correlated', 'uncorrelated', 'correlated', 'correlated', 'uncorrelated', 'correlated', 'uncorrelated', 'uncorrelated', 'uncorrelated', 'uncorrelated', 'uncorrelated', 'correlated', 'uncorrelated', 'uncorrelated', 'correlated', 'uncorrelated', 'correlated', 'correlated', 'uncorrelated', 'correlated', 'uncorrelated', 'uncorrelated', 'uncorrelated', 'uncorrelated', 'uncorrelated', 'uncorrelated', 'correlated', 'uncorrelated', 'correlated', 'uncorrelated',

# Making dataframe for findings

In [26]:
dframe = pd.DataFrame({'Country name':countrys, 'Pearson\'s correlation':correlationp, 'P-value of pearson\'s correlation': pearp, 'Comment on pearson\'s correlation':commentpearson,'Spearsman\'s correlation':correlations,'P-value of spearman\'s correlation':spearp, 'Comment on spearsman\'s correlation':commentspearsman, 'Kendall\'s correlation':correlationk,'P-value of kendall\'s correlation':kendallp, 'Comment on kendall\'s correlation':commentkendall})
dframe

,Country name,Pearson's correlation,P-value of pearson's correlation,Comment on pearson's correlation,Spearsman's correlation,P-value of spearman's correlation,Comment on spearsman's correlation,Kendall's correlation,P-value of kendall's correlation,Comment on kendall's correlation
0,Afghanistan,0.402,0.057,uncorrelated,0.540,0.008,correlated,0.412,0.009,correlated
1,Albania,0.245,0.201,uncorrelated,0.389,0.037,correlated,0.272,0.047,correlated
2,Algeria,-0.114,0.528,uncorrelated,-0.142,0.430,uncorrelated,-0.090,0.482,uncorrelated
3,Andorra,-0.055,0.809,uncorrelated,-0.058,0.798,uncorrelated,-0.041,0.798,uncorrelated
4,Argentina,-0.282,0.147,uncorrelated,-0.304,0.116,uncorrelated,-0.217,0.116,uncorrelated
...,...,...,...,...,...,...,...,...,...,...
110,Uruguay,-0.015,0.941,uncorrelated,-0.028,0.886,uncorrelated,-0.014,0.921,uncorrelated
111,Uzbekistan,0.119,0.596,uncorrelated,0.377,0.084,uncorrelated,0.283,0.073,uncorrelated
112,Venezuela,0.087,0.717,uncorrelated,0.049,0.838,uncorrelated,0.037,0.838,uncorrelated
113,Vietnam,0.304,0.068,uncorrelated,0.268,0.109,uncorrelated,0.193,0.110,uncorrelated


# Making csv for findings

In [27]:
 dframe.to_csv(r'temperaturelatonfindings.csv', index=False)

# plotted using plotly ( But plotly doesn't support dashed grid line so might change later with Matplotlib

In [ ]:
colors = ['olive','olive','olive','olive','olive','olive','olive']

go.Figure([go.Bar(
    x=countrys, y= correlationp,
    text= correlationp,
    marker_color= colors,
    textposition='auto',textfont=dict(
        family="times new roman",
        size = 22,
        color="black"
    )
)]).update_layout( bargap=0.2,
    width= 600, height=800,
    xaxis = dict(title='Country Name'), yaxis = dict(title='Pearson\'s correlation coefficient'),font=dict(family="times new roman", size= 22 , color = "black")
).update_xaxes(showline=True, linewidth=2, linecolor='black',gridcolor='lightgrey',ticks="inside",ticklen=10,tickwidth=2,mirror=True).update_yaxes(showline=True, linewidth=2, linecolor='black',gridcolor='lightgrey',ticks="inside",ticklen=6,tickwidth=2, mirror=True).show(displayModeBar= True)

# Tried with matplotlib but wrong graph

In [ ]:
import matplotlib.pyplot as plt
fig = plt.figure()
ax = fig.add_axes([0,0,1,1])
ax.bar(countrys,correlationp)
plt.show()